# Flask Basics, Lesson 5!!!

## Flask Relationships 

![SegmentLocal](https://media3.giphy.com/media/TFpXAJRtUIxlC/giphy.gif "segment")

There are going to be times in big projects where you want to have more than one model, but these models will likely have relationships with eachother!

Building off the puppies example we've been using, this could be a relationship with the Owner model

An important thing to understand to understand these relationships is Primary Keys and Foreign Keys:

#### Primary Keys:

A unique identifier, something that will truely be different for each item 

#### Foreign Keys:

A primary key in another table!

Using the Puppy and Owner example:

#### Puppy:
ID Tag (Primary Key!)
Puppy Name

#### Owner:
National ID Number (Primary Key)
Owner Name
Puppy ID (Foreign Key)

### Example!!!

![SegmentLocal](https://img.buzzfeed.com/buzzfeed-static/static/2014-10/29/12/enhanced/webdr02/anigif_enhanced-15050-1414600197-24.gif "segment")

In [ ]:
# Models.py
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'+os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAchemy(app)
Migrate(app, db)

class Puppy(db.Model):
     
    __tablename__ = 'puppies'
    
    id = db.Column(db.Integer, primary_key = True)
    name = db.Column(db.Text)
    toys = db.relationship('Toy', backref = 'puppy', lazy = 'dynamic')
    # One to many, one puppy to many toys
    # Lazy specifies how relationship is loaded into database
    owner = db.relationship('owner', backref = 'puppy', uselist = False)
    # one to one, one puppy to own owner
    # Uselist is false because since it is only one owner for one puppy, you will never have a list for this relationship, like you might with toys
    
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        if self.owner:
            return "Puppy name is {self.name} and owner is {self.owner}"
        else:
            return "Puppy is {self.name} and has no owner yet"
        
    def report_toys(self):
        print("Here are my toys:")
        for toy in self.toys:
            print(toy.item_name)
    
class Toy(db.Model):
    
    __tablename__ = 'toys'
    
    id = db.Column(db.Integer, primary_key = True)
    item_name = db.Column(db.Text)
    puppy_id = db.Column(db.Integer, db.ForeignKey('puppies.id'))
    
    def __init__(self, item_name, puppy_id):
        self.item_name = item_name
        self.puppy_id = puppy_id
        
class Owner(db.Model):
    __tablename__ = 'owners'
    
    id = db.Column(db.Integer, primary_key = True)
    name = db.Column(db.Text)
    puppy_id = db.Column(db.Integer, db.ForeignKey('puppies.id'))
    
    def __init__(self, name, puppy_id):
        self.name = name
        self.puppy_id = puppy_id

Source Jose Portilla 

Now we use the code above and create entries!

In [ ]:
#basic.py
from models import db, Puppy, Owner, Toy

rufus = Puppy('Rufus')
#this creates a pup using the self init in the Puppy class, which you pass a name into, 
#therefore we send in 'Rufus'
fido = Puppy('Fido')

db.session.add_all([rufus, fido])
db.session.commit
#adds the puppies to the database

rufus = Puppy.query.filter_by(name = 'Rufus').first()
#This grabs rufus now that it has been saved 

kate = Owner('Kate', rufus.id)
#Creates an owner object, uses self init in owner class
#This takes in a name, and the id of the pet

toy1 = Toy('Chew Toy', rufus.id)
toy2 = Toy('Ball', rufus.id)
#Creating toys for rufus, once again uses Toy class self init

db.session.add_all([kate, toy1, toy2])
db.session.commit()

Source Jose Portilla

## User Authentication!!

![SegmentLocal](https://66.media.tumblr.com/905e3b0e3abfab87071e8c68f869af64/tumblr_p78sscYqBd1qzhbquo2_250.gif "segment")

So far, with everything we've gone over and created, anyone can access it, but like a lot of applications, you may only want registered users!

We can do this with several different tools, the first being:             
### Password Hashing!!

You are going to want to store usernames and passwords for your users, but you never want to store the original password because that is unsecure, and could easily be hacked!

The best way to solve this is a 
### Hash Function

Which is an algorithm that takes in a document, in this case a password, and returns a secure hash based on that document 

Secure because with a hash a person cannot figure out what the password was that got that hash

A really simple example would be using the number of characters in the password as the hash, so:

thispassword --> 12

12345 --> 5

This is not a good hash function because it is unsecure and more than one password could have the same hash 

But we don't have to come up with a hashing algorithm, because there are already secure hashing algorithms for us to use!!!

Two useful libraries:

1) Bcrypt 

2) Werkzeug

This is how to install either:

In [ ]:
pip install flask-bcrypt

pip install Werkzeug

Here is an example for bcrypt!!

In [ ]:
from flask_bcrypt import Bcrypt

bcrypt = Bcrypt()

password = 'supersecretpassword'

hashed_password = bcrypt.generate_password_hash(password)
#This will give you the hashed password, stored in hashed_password

check = bcrypt.check_password_hash(hashed_password, 'wrongpassword')
#This sees if the entered password is correct against the hashed_password
#check is set to either true or false

Example with Werkzeug!!

In [ ]:
from werkzeug.security import generate_password_hash, check_password_hash

hashed_password = generate_password_hash('mypassword')

check = check_password_hash(hashed_pass, 'mypassword')

They work very similarly !!